# Capstone - Generate CLIP embeddings from images

#### Load dataset

In [3]:
import torch
from torchvision import transforms
from transformers import CLIPProcessor, CLIPModel
from PIL import Image

# Load the CLIP model and processor
model_name = "openai/clip-vit-base-patch16"
device = "cuda"
model = CLIPModel.from_pretrained(model_name).to(device)
processor = CLIPProcessor.from_pretrained(model_name)

/opt/conda/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/opt/conda/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c104cuda9SetDeviceEi'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [4]:
# Preprocess the image
def preprocess_image(image_path):
    image = Image.open(image_path).convert("RGB")
    image = transforms.Resize((224, 224))(image)
    image = transforms.ToTensor()(image)
    return image.unsqueeze(0)

# Encode the image
def encode_image(image_path):
    # image = preprocess_image(image_path)
    inputs = processor(text=[""],images=image_path, return_tensors="pt", padding=True)
    inputs = {k: v.to("cuda") for k, v in inputs.items()}  # 
    outputs = model(**inputs)
    img_embedding = outputs.image_embeds
    return img_embedding


In [5]:
import os
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from PIL import Image
from tqdm import tqdm

class CustomImageDataset(torch.utils.data.Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = [os.path.join(root_dir, f) for f in os.listdir(root_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, img_path

# Define the image transformation
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

# Create an instance of the custom dataset
image_folder = "era/train2017"
emb_path = "era/img_emb"
custom_dataset = CustomImageDataset(root_dir=image_folder, transform=transform)

# Create a data loader with batch size
batch_size = 32  # Adjust the batch size based on memory constraints
data_loader = DataLoader(custom_dataset, batch_size=batch_size, shuffle=False)

# image_paths = []
# Iterate through the data loader to process images in batches
for ix, (batch_images, batch_image_paths) in tqdm(enumerate(data_loader)):
    # Process the images in the current batch
    image_embedding = encode_image(batch_images)
    # image_paths.extend(batch_image_paths)
    torch.save(image_embedding, os.path.join(emb_path, f'embeddings_batch{ix}.pth'))


0it [00:00, ?it/s]It looks like you are trying to rescale already rescaled images. If the input images have pixel values between 0 and 1, set `do_rescale=False` to avoid rescaling them again.
3697it [6:08:59,  5.99s/it]


In [ ]:
tensor_list = []
for i in range(3697):
    file_path = os.path.join(emb_path, f"embeddings_batch{i}.pth")
    tensor_list.append(torch.load(file_path))
embs = torch.concat(tensor_list, axis=0)
torch.save(embs,"img_embeddings.pth")\

In [ ]:
img_emb = torch.load("img_embeddings.pth").unsqueeze(1).to("cpu")

with open("./image_names.json", 'r') as file:
    image_names = json.load(file)
imgEmbDict = dict(zip(image_names, img_emb))

In [ ]:
torch.save(imgEmbDict, 'img_embeddings_dict.pth')